<figure style="float:right">
    <img
    src="./data/callat_logo.png"
    width="150"
    alt="CalLat logo"
    /img>
</figure>

# Jupyter notebook for CalLat gA project 

## Import libraries

In [1]:
import pandas as pd
pd.options.display.max_rows = 16
import numpy as np
import scipy as sp
import scipy.stats as stats
%matplotlib inline
import matplotlib as mpl
import lsqfit
import gvar as gv
import callat_ga_lib as xlib
import sys
print("python version:", sys.version)
print("pandas version:", pd.__version__)
print("numpy  version:", np.__version__)
print("scipy  version:", sp.__version__)
print("mpl    version:", mpl.__version__)
print("lsqfit version:", lsqfit.__version__)
print("gvar   version:", gv.__version__)

python version: 3.6.1 |Anaconda 4.4.0 (x86_64)| (default, May 11 2017, 13:04:09) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]
pandas version: 0.20.1
numpy  version: 1.12.1
scipy  version: 0.19.0
mpl    version: 2.0.2
lsqfit version: 9.1.3
gvar   version: 8.3.2


## Define analysis parameters
* `switches['ensembles']` | list of strings
    * select the ensembles that are used to perform the extrapolation
    * the three rows correspond to the 0.15, 0.12, and 0.09 fm ensembles that are available
* `switches['ansatz']` | dictionary
    * define the fit ansatz for the extrapolation
    * `['type']` | string: chooses between a Taylor expansion or Baryon Xpt
        * Taylor expansion only includes even powers of ε<sub>π</sub>
    * `['truncation']` | integer: is an integer n corresponding to the order of ε<sub>π</sub><sup>n</sup>
    * `['FV']` | boolean: True turns on NLO FV corrections for both Baryon Xpt and Taylor
    * `['xsb']` | boolean: True turns on am<sub>π</sub> term for Baryon Xpt
    * `['alpha']` | boolean: True turns on α<sub>s</sub>a<sup>2</sup> for Baryon Xpt

In [2]:
switches = dict()
# Ensembles used in extrapolation
switches['ensembles'] = [
    'l1648f211b580m0217m065m838','l2464f211b600m0170m0509m635','l3264f211b630m0124m037m440',
    'l1648f211b580m0166m065m838','l2464f211b600m0130m0509m635','l3264f211b630m00945m037m440',
    'l1648f211b580m013m065m838','l2464f211b600m0102m0509m635','l3296f211b630m0074m037m440',
    'l2448f211b580m0064m0640m828','l3264f211b600m00507m0507m628','l4896f211b630m00363m0363m430',
    'l2464f211b600m00507m0507m628','l4064f211b600m00507m0507m628',
    'l3248f211b580m00235m0647m831','l4864f211b600m00184m0507m628'
    ]

switches['ansatz'] = dict()
### Type of fit: 'xpt_N', 'taylor_N', 'linear_N', 'constant_N', 'xpt-full_4', 'xpt-doublelog_4', 'xpt-delta_N'
switches['ansatz']['type'] = ['xpt_3','xpt_4','taylor_2','taylor_4','linear_2','linear_4']
#switches['ansatz']['type'] = ['taylor_2','taylor_4']
switches['ansatz']['FV'] = True # True turns on NLO FV correction
switches['ansatz']['xsb'] = False # True turns on O(a) discretization
switches['ansatz']['alpha'] = False # True turns on O(alpha_s a^2) discretization
### plot tools 
switches['plot'] = dict()
switches['plot']['chiral'] = True
switches['plot']['continuum'] = True
switches['plot']['FV'] = True
switches['plot']['model_avg_histogram'] = True
# For the switches below to work
# the corresponding plot above must be True
switches['plot']['model_avg_chiral'] = True
switches['plot']['model_avg_cont'] = True
switches['plot']['model_avg_fv'] = True
# Save figs to local directory?
switches['save_figs'] = True
# inflation of prior widths
# multiplicative factor for prior width of epsilon_delta
switches['eps_delta_sig'] = 0.05
switches['axial_sig'] = 0.3333
# multiplicative factor for NxLO LECs
p_lo = 1.
p_nlo = 1.
p_nnlo = 1.
p_nnloa = 1.

## Define priors and PDG values
`gvar` datatype has the form `gv.gvar(mean, standard deviation)`

[gvar documentation](https://github.com/gplepage/gvar/blob/master/doc/gvar.pdf)

In [3]:
priors = dict()
# Xpt priors
priors['xpt'] = dict()
priors['xpt']['g0'] = gv.gvar(1.0, p_lo*1.0) # LO LEC
priors['xpt']['a1'] = gv.gvar(0.0, 1E-3) # DWF order a discretization
priors['xpt']['c2'] = gv.gvar(0.0, p_nlo*50.0) # NLO counterterm epi^2
priors['xpt']['c3'] = gv.gvar(0.0, p_nnlo*50.0) # NNLO LEC epi^3
priors['xpt']['a2'] = gv.gvar(0.0, p_nlo*50.0) # NLO a^2
priors['xpt']['s2'] = gv.gvar(0.0, 1.0) # NLO alpha_s a^2
priors['xpt']['a4'] = gv.gvar(0.0, p_nnloa*p_nnlo*1.0) # NNNLO a^4
priors['xpt']['b4'] = gv.gvar(0.0, p_nnlo*1.0) # NNNLO a^2 epi^2
priors['xpt']['c4'] = gv.gvar(0.0, p_nnlo*10.0) # NNNLO epi^4
priors['xpt']['gm4'] = gv.gvar(0.0, 50.0) # NNNLO log term
priors['xpt']['gnd0'] = gv.gvar(-6./5*1.2,switches['axial_sig']*6/5*1.2) # delta LECs
priors['xpt']['gdd0'] = gv.gvar(-9./5*1.2,switches['axial_sig']*9/5*1.2) # delta LECs
# taylor priors
priors['taylor'] = dict()
priors['taylor']['g0'] = gv.gvar(1.5, p_nlo*1.5) # FV coefficient
priors['taylor']['c0'] = gv.gvar(1.0, p_lo*10.0) # constant
priors['taylor']['c2'] = gv.gvar(0.0, p_nlo*10.0) # epi^2
priors['taylor']['a2'] = gv.gvar(0.0, p_nlo*10.0) # a^2
priors['taylor']['c4'] = gv.gvar(0.0, p_nnlo*10.0) # epi^4
priors['taylor']['a4'] = gv.gvar(0.0, p_nnloa*p_nnlo*1.0) # a^4
priors['taylor']['b4'] = gv.gvar(0.0, p_nnlo*1.0) # a^2 epi^2
# linear priors
priors['linear'] = dict()
priors['linear']['g0'] = gv.gvar(1.5, p_nlo*1.5) # FV coefficient
priors['linear']['c0'] = gv.gvar(1.0, p_lo*10.0) # constant
priors['linear']['c2'] = gv.gvar(0.0, p_nlo*10.0) # epi
priors['linear']['a2'] = gv.gvar(0.0, p_nlo*50.0) # a^2
priors['linear']['a4'] = gv.gvar(0.0, p_nnloa*p_nnlo*1.0) # a^4
priors['linear']['c4'] = gv.gvar(0.0, p_nnlo*1.0) # epi^2
# constant priors
priors['constant'] = dict()
priors['constant']['g0'] = gv.gvar(1.5, p_nlo*1.5) # FV coefficient
priors['constant']['c0'] = gv.gvar(1.0, p_lo*10.0) # constant
priors['constant']['a2'] = gv.gvar(0.0, p_nlo*10.0) # a^2
priors['constant']['a4'] = gv.gvar(0.0, p_nnloa*p_nnlo*1.0) # a^4

# Physical parameters from PDG
phys_params = dict()
# http://pdg.lbl.gov/2016/tables/rpp2016-tab-mesons-light.pdf
phys_params['mpi'] = gv.gvar(139.57018, 0.00035) # mpi +/- [MeV]
# http://pdg.lbl.gov/2016/reviews/rpp2016-rev-pseudoscalar-meson-decay-cons.pdf
phys_params['fpi'] = gv.gvar(130.2, 1.7) # fpi + ['MeV']
# http://pdg.lbl.gov/2017/listings/rpp2017-list-Delta-1232.pdf
phys_params['Delta'] = gv.gvar(293, 2) # Delta(1232) Breit Wigner Mass

## Import data
[pandas dataframe documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)

In [4]:
# import correlator bootstraps
gadf = pd.read_csv('./data/github_ga_v2.csv')
gadf.groupby('ensemble').describe()[['ga','epi','mpil']]

KeyError: "['ga' 'epi' 'mpil'] not in index"

In [ ]:
# import HISQ parameters
hqdf = pd.read_csv('./data/hisq_params.csv')
hqdf

## Format data for analysis

In [ ]:
data = xlib.format_data(switches, gadf, hqdf)

## Chiral-continuum fit
[lsqfit documentation](https://github.com/gplepage/lsqfit/blob/master/doc/lsqfit.pdf)

In [ ]:
result = xlib.fit_data(switches, priors, data, phys_params)
for a in switches['ansatz']['type']:
    print(result[a]['fit'])
    print("%s physical point result:" %a, result[a]['phys']['result'])
    print('%s, %s, %s, %s' %(result[a]['phys']['result'].mean, result[a]['phys']['result'].sdev, result[a]['fit'].chi2/result[a]['fit'].dof, result[a]['fit'].logGBF))

## Systematic error budget
The total uncertainty here differs from the paper because uncertainty from finite volume and isospin breaking have yet to be included. These two systematic uncertainties are estimated independently and added in quadrature after this analysis.

In the paper, the input uncertainty is absorbed into the statistical uncertainty.

In [ ]:
error = xlib.error_budget(switches,result)
for a in switches['ansatz']['type']:
    print(a)
    print(pd.DataFrame.from_dict(error[a]['pct']).rename(index={0: 'pct. uncertainty'})[['stat','chiral','disc','input','total']])
    print('')

## Plot results
* chiral extrapolation
* convergence of extrapolation
* continuum extrapolation
* infinite volume extrapolation

In [ ]:
# Chiral extrapolation and series convergence
if switches['plot']['chiral']:
    Plot = xlib.plot_chiral_fit()
    r_chiral = Plot.plot_chiral(switches,data,result)
    mpl.pyplot.show()

In [ ]:
# Continuum extrapolation
if switches['plot']['continuum']:
    Plot = xlib.plot_chiral_fit()
    r_cont = Plot.plot_continuum(switches,data,result)
    mpl.pyplot.show()

In [ ]:
# Infinite volume extrapolation at 220 MeV 0.12 fm
if switches['plot']['FV']:
    Plot = xlib.plot_chiral_fit()
    r_fv = Plot.plot_volume(switches,data,result)
    mpl.pyplot.show()

## Model averaging
Model average with Bayes Factors provides more robust prediction than choosing any single result.

From marginalizing over models $M_k$ for $k \in \{\textrm{models}\}$ we get the averaged posterior distribution to be:

$P(g_A | \textrm{data}) = \sum_k P(g_A | M_k, \textrm{data})P(M_k|\textrm{data})$

where

$P(M_i|\textrm{data}) = \frac{\textrm{BF}_i P(M_i)}{\sum_k \textrm{BF}_k P(M_k)} = \frac{\textrm{BF}_i}{\sum_k \textrm{BF}_k}$

where the second equality is true if there are no a priori preferred models.

In [ ]:
# read Bayes Factors
logGBF_list = []
for a in switches['ansatz']['type']:
    logGBF_list.append(result[a]['fit'].logGBF)
# initiate a bunch of parameters
# gA
gA = 0
gA_lst = []
# weights
w_lst = []
wd = dict()
# p. dist. fcn
pdf = 0
pdfdict = dict()
# c. dist. fcn.
cdf = 0
cdfdict = dict()
# for plotting
x = np.linspace(1.222,1.352,13000)
for a in switches['ansatz']['type']:
    r = result[a]['phys']['result']
    w = 1/sum(np.exp(np.array(logGBF_list)-result[a]['fit'].logGBF))
    wd[a] = w
    w_lst.append(w)
    gA += w*r
    gA_lst.append(r.mean)
    p = stats.norm.pdf(x,r.mean,r.sdev)
    pdf += w*p
    pdfdict[a] = w*p
    c = stats.norm.cdf(x,r.mean,r.sdev)
    cdf += w*c
    cdfdict[a] = w*c
gA_lst = np.array(gA_lst)
w_lst = np.array(w_lst)
model_var = np.sum(w_lst*(gA_lst - gA.mean)**2)
# extra uncertainties in quadrature
fverror = 0.000158
isospin = 0.000509
final_error = np.sqrt(gA.sdev**2 + fverror**2 + isospin**2)
print('  weight : model')
for k in wd:
    print(' %f: %s' %(wd[k],k))
print('gA = %f +- %f +- %f' %(gA.mean,final_error,np.sqrt(model_var)))
print ('percent uncertainty: %f +- %f' %(final_error/gA.mean*100,np.sqrt(model_var)/gA.mean*100))
print('gA = %f, %f' %(gA.mean,np.sqrt(final_error**2+model_var)))
print('percent uncertainty: %f' %(np.sqrt(final_error**2+model_var)/gA.mean*100))
model_budget = xlib.modelavg_error(switches,result,gA)
print("Error budget from extrapolation")
print(pd.DataFrame.from_dict(model_budget['pct']).rename(index={0: 'pct. uncertainty'})[['stat','chiral','disc','total']])

In [ ]:
if switches['plot']['model_avg_histogram']:
    Plot = xlib.plot_chiral_fit()
    Plot.plot_histogram(switches,x,pdf,pdfdict,cdf)

In [ ]:
if switches['plot']['model_avg_chiral']:
    Plot = xlib.plot_chiral_fit()
    Plot.model_avg_chiral(switches,phys_params,wd,r_chiral)

In [ ]:
if switches['plot']['model_avg_cont']:
    Plot = xlib.plot_chiral_fit()
    Plot.model_avg_cont(switches,wd,r_cont)

In [ ]:
if switches['plot']['model_avg_fv']:
    Plot = xlib.plot_chiral_fit()
    Plot.model_avg_fv(switches,wd,r_fv)

<center>
    <span style="color: black; font-family: Helvetica; font-size: 2em">
        These calculations are made possible by
    </span>
</center>

<figure>
    <img src='./data/incite_logo.png' width='200' />
</figure>
<figure>
    <img src='./data/olcf_logo.png' width='240' />
</figure>
<figure>
    <img src='./data/llnl_logo.png' width='550' />
</figure>